## Xcube with H5NetCDF

In [1]:
import xarray as xr
from xcube.core.gridmapping import GridMapping
from xcube.core.resampling import resample_in_space

In [2]:
def configure_fs_auth():
    import earthaccess
    import s3fs

    auth = earthaccess.login()
    s3_credentials = auth.get_s3_credentials("PODAAC")
    fs = s3fs.S3FileSystem(
        anon=False,
        key=s3_credentials["accessKeyId"],
        secret=s3_credentials["secretAccessKey"],
        token=s3_credentials["sessionToken"],
    )
    return fs

In [3]:
def warp_resample():
    bucket = "podaac-ops-cumulus-protected"
    input_uri = "MUR-JPL-L4-GLOB-v4.1/20020601090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc"
    variable = "analysed_sst"
    src = f"s3://{bucket}/{input_uri}"
    dstSRS = "EPSG:3857"
    srcSRS = "EPSG:4326"
    width = height = 256
    te = [
        -20037508.342789244,
        -20037508.342789244,
        20037508.342789244,
        20037508.342789244,
    ]
    xres = (te[2] - te[0]) / width
    yres = (te[3] - te[1]) / width
    fs = configure_fs_auth()
    fsspec_caching = {
        "cache_type": "none",
    }
    with fs.open(src, **fsspec_caching) as f:
        ds = xr.open_dataset(f, engine="h5netcdf", chunks={})[[variable]]
        source_gm = GridMapping.from_coords(
            x_coords=ds.lat, y_coords=ds.lon, crs=srcSRS
        )
        target_gm = GridMapping.regular(
            size=[height, width], xy_min=[te[0], te[1]], xy_res=[xres, yres], crs=dstSRS
        )
        return resample_in_space(ds, source_gm=source_gm, target_gm=target_gm)

In [4]:
%%time
if __name__ == "__main__":
    da = warp_resample()

/opt/conda/lib/python3.11/site-packages/dask/core.py:133: RuntimeWarning: invalid value encountered in subtract
  return func(*(_execute_task(a, cache) for a in args))
/opt/conda/lib/python3.11/site-packages/dask/core.py:133: RuntimeWarning: invalid value encountered in subtract
  return func(*(_execute_task(a, cache) for a in args))
/opt/conda/lib/python3.11/site-packages/dask/array/reductions.py:1113: RuntimeWarning: All-NaN slice encountered
  vals = func(x, axis=arg_axis, keepdims=True)


OverflowError: cannot convert float infinity to integer